In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('test_data.csv')
data = data[data['role'] == 'manager']
data.head()

,dlg_id,line_n,role,text
1,0,1,manager,Алло здравствуйте
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
5,0,5,manager,Угу ну возможно вы рассмотрите и другие вариан...
8,0,8,manager,Угу а на что вы обращаете внимание при выборе
11,0,11,manager,Что для вас приоритет


In [3]:
dict_rus = {'ADJ': 'прилагательное',
    'ADP': 'приставка',
    'ADV': 'наречие',
    'AUX': 'вспомогательный',
    'CCONJ': 'сочинительный союз' ,
    'DET': 'определение',
    'INTJ': 'междометие',
    'NOUN': 'существительное',
    'NUM': 'цифра',
    'PART': 'частица',
    'PRON': 'местоимение',
    'PROPN': 'имя собственное',
    'PUNCT': 'знаки препинания' ,
    'SCONJ': 'подчинительный союз',
    'SYM': 'символ',
    'VERB': 'глагол',
    'X': 'другое'}

In [143]:
text = data['text'][111]
text

'Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается'

In [218]:
import spacy
from spacy.displacy import serve as ss
nlp = spacy.load("ru_core_news_sm")
import re
import nltk
import pymorphy2


class NLP:
    morf = pymorphy2.MorphAnalyzer()
    def __init__(self, origin):  # lemma, name_manager, greeting, farewell, criterion):
        self.origin = origin
        self.lemma = self.func(self.origin)
        self.name_maneger = self.re_func(self.lemma)
        self.greeting = self.greet(self.origin)
        self.farewell = self.farel(self.origin)
        self.company = self.compan(self.lemma)
        
        
    def func(self, text):
        doc = nlp(text)
        return ' '.join([i.lemma_ for i in doc])
    
    def re_func(self, text):
        if re.search(r'меня \w+ звать', text):
            return re.findall(r'меня \w+ звать', text)[0].split()[-2]
        elif re.search(r'меня звать \w', text):
            return re.findall(r'меня звать \w+', text)[0].split()[-1]
        elif re.search(r'это \w+', text):
            name = re.findall(r'это \w+', text)[0].split()[-1]
            if 'Name' in morf.parse(name)[0].tag:
                return name
            else:
                return None
        else:
            return None
    
    def greet(self, text):
        if re.search(r'здравств|добрый день|добрый вечер|доброе утро', text):
            return re.findall(r'здравств.*|добрый день|добрый вечер|доброе утро', text)[0]
        else:
            return False
    
    def farel(self, text):
        if re.search(r'до свидан|всего хорош|всего добр', text):
            return re.findall(r'до свидан.*|всего хорош.*|всего добр.*', text)[0]
        else:
            return False   
    
    def compan(self, text):
        if re.search(r'компания', text):
            name = re.findall(r'компания \w+ \w+', text)[0].split()
            if 'NOUN' in morf.parse(name[1])[0].tag:
                return name[1]
            elif 'NOUN' in morf.parse(name[2])[0].tag:
                return ' '.join(name[1:])
            else:
                return None
        else:
            return None

In [219]:
a = NLP('алло здравстуйте')

In [220]:
a.name_maneger

In [223]:
dict_s = {}

for i in data['dlg_id'].unique():
    data_iter = data[data['dlg_id'] == i]
    dict_s[f'dlg_id_{i}'] = {'name_maneger': 'Не представился', 
                             'greeting': False, 'farewell': False,
                             'criterion':False, 'company': 'Не названа'}
    for string in data_iter['text']:
        out = NLP(string)
        if out.name_maneger != None:
            dict_s[f'dlg_id_{i}']['name_maneger'] = out.name_maneger
            
        if out.greeting != False:
            dict_s[f'dlg_id_{i}']['greeting'] = out.greeting
        if out.farewell != False:
            dict_s[f'dlg_id_{i}']['farewell'] = out.farewell
        if dict_s[f'dlg_id_{i}']['greeting'] != False and dict_s[f'dlg_id_{i}']['farewell'] != False:
            dict_s[f'dlg_id_{i}']['criterion'] = True
        if out.company != None:
            dict_s[f'dlg_id_{i}']['company'] = out.company

In [224]:
dict_s

{'dlg_id_0': {'name_maneger': 'ангелина',
  'greeting': 'здравствуйте',
  'farewell': 'до свидания',
  'criterion': True,
  'company': 'диджитал бизнес'},
 'dlg_id_1': {'name_maneger': 'ангелина',
  'greeting': 'здравствуйте',
  'farewell': 'всего хорошего',
  'criterion': True,
  'company': 'диджитал бизнес'},
 'dlg_id_2': {'name_maneger': 'ангелина',
  'greeting': 'здравствуйте',
  'farewell': False,
  'criterion': False,
  'company': 'диджитал бизнес'},
 'dlg_id_3': {'name_maneger': 'максим',
  'greeting': 'добрый день',
  'farewell': 'всего доброго',
  'criterion': True,
  'company': 'китобизнес'},
 'dlg_id_4': {'name_maneger': 'Не представился',
  'greeting': False,
  'farewell': 'до свидания',
  'criterion': False,
  'company': 'Не названа'},
 'dlg_id_5': {'name_maneger': 'анастасия',
  'greeting': False,
  'farewell': 'до свидания хорошего вечера',
  'criterion': False,
  'company': 'Не названа'}}

In [225]:
number_dlg = 0

for i in data[data['dlg_id'] == number_dlg]['text']:
    print(i)


Алло здравствуйте
Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается
Угу ну возможно вы рассмотрите и другие варианты видите это хорошая практика сравнивать
Угу а на что вы обращаете внимание при выборе
Что для вас приоритет
Ну у вас срок заканчивается поэтому мы набрали + ко всему
А так нет не только поэтому просто я обратила внимание и вы пользуетесь виджетами мы там с вами сотрудничали
А если вы 19 являетесь то лучше то идти бесплатно
Ага хорошо
Индивидуальным поэтому не все то есть сотрудничают с кем то сейчас да кому то например не с кем сейчас справляться кто то сравнивал
Да мы услышали вас спасибо за рекомендации
Вот понимаю да но дело в том что в основном да за месяц
Необходимо обзванивать
Хорошо я данную информацию тогда зафиксирую передам будем иметь ввиду
И вам спасибо большое за обратную связь
Всего хорошего до свидания
